# See how many points are inside and outside of the labels, before and after filtering

In [4]:
# Libraries
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path

In [5]:
# List of paths to dataset
BASE_DIR = '../../data/eval_data'
ARCS_DIR = 'arcs'
ARCS_FILTERED_DIR = 'arcs_azimuth_filtered'
ARCS_LABEL_FILTERED_DIR = 'arcs_label_filtered'
ARCS_LABEL_DIR = 'labels'

In [6]:
# Evaluate filter (before eval script)
BASE_DIR = '../../data'
ARCS_DIR = 'velodyne_points'
ARCS_FILTERED_DIR = 'filtered_points'
ARCS_LABEL_FILTERED_DIR = 'label_filtered_points'
ARCS_LABEL_DIR = 'pillar_pred_format_labels'

In [7]:
def point_is_in_an_roi_box(point, roi_box_list):
    # Extract the point coordinates
    x, y, z = point
    
    for bbox in roi_box_list:
        # Extract the box parameters
        # dz, dy, dx, cx, cy, cz, yaw = bbox
        cx, cy, cz, dx, dy, dz, yaw = bbox
        
        # Create the rotation matrix for the yaw angle
        cos_yaw = np.cos(-yaw)  # Negative for the inverse rotation
        sin_yaw = np.sin(-yaw)
        
        rotation_matrix = np.array([
            [cos_yaw, -sin_yaw, 0],
            [sin_yaw,  cos_yaw, 0],
            [0,       0,       1]
        ])
        
        # Translate the point to the box's coordinate frame
        translated_point = np.array([x - cx, y - cy, z - cz])
        
        # Rotate the point to align with the box
        rotated_point = rotation_matrix.dot(translated_point)
        
        # Check if the point is within the box dimensions
        half_lengths = [dz / 2, dy / 2, dx / 2]
        in_box = all([
            -half_lengths[i] <= rotated_point[i] <= half_lengths[i] for i in range(3)
        ])
    
        if in_box:
            return True
    
    return False

In [8]:
def get_roi_boxes(label_file):
    bboxes = []
    with open(label_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = [float(value) for value in parts[8:15]]  # Extract the bounding box dimensions and location            
            bboxes.append(bbox)
    return np.array(bboxes)

In [9]:
def is_label_point(point, label_file_path):
    list_roi_boxes = get_roi_boxes(label_file_path)
    if point_is_in_an_roi_box(point, list_roi_boxes):
        return True
    return False

In [10]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [11]:
def get_frame_point_statistics(data_file_path, label_file_path):
    # Running totals
    total_points = 0
    label_points = 0

    points_df = convert_to_dataframe(data_file_path)
    
    # For each point in the data
    for index, row in points_df.iterrows():
    # for index, row in points_df.iloc[:100].iterrows():
        # Increment the total
        total_points += 1
        point = (row['x'], row['y'], row['z'])
        # If the point is a label point
        if is_label_point(point, label_file_path):
            # Increment the label total
            label_points += 1

    return total_points, label_points

In [12]:
test_files = Path(BASE_DIR, ARCS_FILTERED_DIR)
files = [f for f in os.listdir(test_files) if f.endswith('.bin')]
random.shuffle(files)
files = files[:100]
print(files)

['000372.bin', '004798.bin', '000795.bin', '004569.bin', '000014.bin', '001938.bin', '004042.bin', '005372.bin', '002898.bin', '001764.bin', '002758.bin', '005913.bin', '004603.bin', '002442.bin', '005325.bin', '001282.bin', '001687.bin', '002830.bin', '003173.bin', '002804.bin', '004228.bin', '003460.bin', '001313.bin', '006113.bin', '005324.bin', '002837.bin', '004014.bin', '006288.bin', '004108.bin', '002075.bin', '003843.bin', '000706.bin', '000091.bin', '001421.bin', '002808.bin', '002720.bin', '000875.bin', '005002.bin', '006227.bin', '000783.bin', '004374.bin', '001965.bin', '002046.bin', '003418.bin', '003465.bin', '003849.bin', '002874.bin', '005241.bin', '001106.bin', '002239.bin', '003861.bin', '005718.bin', '004343.bin', '002482.bin', '001280.bin', '005464.bin', '005619.bin', '000167.bin', '002244.bin', '004596.bin', '002710.bin', '002264.bin', '002797.bin', '000156.bin', '002393.bin', '006281.bin', '002688.bin', '002844.bin', '000147.bin', '000674.bin', '001878.bin', '0017

In [13]:
def get_dataset_point_statistics(data_path, label_path):
    # Running totals
    num_frames = 0
    total_num_points = 0
    total_num_label_points = 0
    
    # Get just the file names
    # files = [f for f in os.listdir(data_path) if f.endswith('.bin')]
    
    # for filename in files[:10]:
    for filename in files:
        # Get file index
        file_id, extension = os.path.splitext(filename)
        print(file_id, end=' ')
        
        # Append file name to locations
        data_file_path = Path(data_path, filename)
        label_filename = file_id + '.txt'
        label_file_path = Path(label_path, label_filename)

        # Get the number of points, and the number of points inside the labels
        num_frame_points, num_frame_label_points = get_frame_point_statistics(data_file_path, label_file_path)
        # Add to running total
        num_frames += 1
        total_num_points += num_frame_points
        total_num_label_points += num_frame_label_points

    # return total_num_points / num_frames, total_num_label_points / num_frames
    return num_frames, total_num_points, total_num_label_points

In [14]:
def evaluate_filter(dir, dataset_name):
    # Create directory path
    data_path = Path(BASE_DIR, dir)
    label_path = Path(BASE_DIR, ARCS_LABEL_DIR)
    print(data_path)

    # Get the dataset statistics
    num_frames, total_num_points, total_num_label_points = get_dataset_point_statistics(data_path, label_path)

    results = {
        'dataset_name': dataset_name,
        'num_frames': num_frames,
        'total_num_points': total_num_points,
        'total_num_label_points': total_num_label_points,
        'total_num_non_label_points': total_num_points - total_num_label_points,
        'avg_frame_pts': total_num_points / num_frames,
        'avg_frame_label_pts': total_num_label_points / num_frames,
        'avg_frame_non_label_pts': (total_num_points - total_num_label_points) / num_frames
    }

    return results

### Run datasets through filter evaluator, print results

In [16]:
results = evaluate_filter(ARCS_DIR, 'ARCS')

..\..\data\velodyne_points
000372 004798 000795 004569 000014 001938 004042 005372 002898 001764 002758 005913 004603 002442 005325 001282 001687 002830 003173 002804 004228 003460 001313 006113 005324 002837 004014 006288 004108 002075 003843 000706 000091 001421 002808 002720 000875 005002 006227 000783 004374 001965 002046 003418 003465 003849 002874 005241 001106 002239 003861 005718 004343 002482 001280 005464 005619 000167 002244 004596 002710 002264 002797 000156 002393 006281 002688 002844 000147 000674 001878 001707 004344 003819 004352 002850 002608 000270 000256 002975 002834 002711 005684 005659 002120 002363 005350 002434 001488 001227 001439 001553 004945 005960 004457 000106 002765 005818 000828 002123 

In [17]:
filter_results = evaluate_filter(ARCS_FILTERED_DIR, 'Filtered ARCS')

..\..\data\filtered_points
000372 004798 000795 004569 000014 001938 004042 005372 002898 001764 002758 005913 004603 002442 005325 001282 001687 002830 003173 002804 004228 003460 001313 006113 005324 002837 004014 006288 004108 002075 003843 000706 000091 001421 002808 002720 000875 005002 006227 000783 004374 001965 002046 003418 003465 003849 002874 005241 001106 002239 003861 005718 004343 002482 001280 005464 005619 000167 002244 004596 002710 002264 002797 000156 002393 006281 002688 002844 000147 000674 001878 001707 004344 003819 004352 002850 002608 000270 000256 002975 002834 002711 005684 005659 002120 002363 005350 002434 001488 001227 001439 001553 004945 005960 004457 000106 002765 005818 000828 002123 

In [18]:
label_filter_results = evaluate_filter(ARCS_LABEL_FILTERED_DIR, 'Label Filtered ARCS')

..\..\data\label_filtered_points
000372 004798 000795 004569 000014 001938 004042 005372 002898 001764 002758 005913 004603 002442 005325 001282 001687 002830 003173 002804 004228 003460 001313 006113 005324 002837 004014 006288 004108 002075 003843 000706 000091 001421 002808 002720 000875 005002 006227 000783 004374 001965 002046 003418 003465 003849 002874 005241 001106 002239 003861 005718 004343 002482 001280 005464 005619 000167 002244 004596 002710 002264 002797 000156 002393 006281 002688 002844 000147 000674 001878 001707 004344 003819 004352 002850 002608 000270 000256 002975 002834 002711 005684 005659 002120 002363 005350 002434 001488 001227 001439 001553 004945 005960 004457 000106 002765 005818 000828 002123 

In [19]:
results['percent_of_non_label_pts_retained'] = results['total_num_non_label_points'] / results['total_num_non_label_points']
results['percent_of_label_pts_retained'] = results['total_num_label_points'] / results['total_num_label_points']

In [20]:
filter_results['percent_of_non_label_pts_retained'] = filter_results['total_num_non_label_points'] / results['total_num_non_label_points']
filter_results['percent_of_label_pts_retained'] = filter_results['total_num_label_points'] / results['total_num_label_points']

In [21]:
label_filter_results['percent_of_non_label_pts_retained'] = label_filter_results['total_num_non_label_points'] / results['total_num_non_label_points']
label_filter_results['percent_of_label_pts_retained'] = label_filter_results['total_num_label_points'] / results['total_num_label_points']

In [41]:
results_df = pd.DataFrame([results, filter_results, label_filter_results])
columns_to_display = ['dataset_name',
                      'avg_frame_non_label_pts',
                      'avg_frame_label_pts',
                      'percent_of_non_label_pts_retained',
                      'percent_of_label_pts_retained']
display(results_df[columns_to_display])

,dataset_name,avg_frame_non_label_pts,avg_frame_label_pts,percent_of_non_label_pts_retained,percent_of_label_pts_retained
0,ARCS,51683.86,856.40,1.00000,1.000000
1,Filtered ARCS,4364.72,557.72,0.08445,0.651238
2,Label Filtered ARCS,0.00,856.40,0.00000,1.000000
